In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

import subprocess


In [5]:
def splitter(test):
    tmp = test.split()
    tst = [tmp[4]]
    tst += tmp[8].split("/")[1].split("_")
    return tst

def convert(s):
    if np.can_cast("193M", to="int"):
        return int(s)
    x, u, b = s[:-1], s[-1], 1000
    return float(x) * {"K":b, "M":b**2, "G":b**3}[u]


In [9]:
temp = subprocess.getoutput("ls -lah keys/80_1024*/b*") \
     + subprocess.getoutput("ls -lah keys/std62_2048*/b*")

df = pd.DataFrame(
    [splitter(line) for line in temp.split("\n")], columns=["disk","bs","poly","dim","base_log","level"])
df["disk"] = df["disk"].apply(convert)/1e6

#df.head()
df.pivot_table(values="disk", index="level", columns=["poly", "base_log"], aggfunc="mean")


poly       1024                                2048                          \
base_log      3      4      5      6      7       3       4       5       6   
level                                                                         
3         193.0  193.0  193.0  193.0  193.0     NaN   769.0   769.0   769.0   
4         257.0  257.0  257.0  257.0  257.0  1100.0  1100.0  1100.0  1100.0   
5         321.0  321.0  321.0  321.0  321.0  1300.0  1300.0  1300.0  1300.0   
6         385.0  385.0  385.0  385.0  385.0  1600.0  1600.0  1600.0  1600.0   
7         449.0  449.0  449.0  449.0  449.0  1800.0  1800.0  1800.0  1800.0   
8         513.0  513.0  513.0  513.0  513.0  2100.0  2100.0  2100.0  2100.0   
9         577.0  577.0  577.0  577.0  577.0  2300.0  2300.0  2300.0  2300.0   

poly              
base_log       7  
level             
3          769.0  
4         1100.0  
5         1300.0  
6         1600.0  
7         1800.0  
8         2100.0  
9         2300.0

In [10]:
temp = subprocess.getoutput("ls -lah keys/80_1024*/sk*")\
     + subprocess.getoutput("ls -lah keys/std62_2048*/sk*")

df = pd.DataFrame(
    [splitter(line) for line in temp.split("\n")], columns=["disk","bs","poly","dim","base_log","level"])
df["disk"] = df["disk"].apply(convert)/1e3

#df.head()
df.pivot_table(values="disk", index="level", columns=["poly", "base_log"])


poly      1024                          2048                        
base_log     3     4     5     6     7     3     4     5     6     7
level                                                               
3         2.15  2.15  2.15  2.15  2.15  4.20  4.15  4.15  4.15  4.15
4         2.15  2.15  2.15  2.15  2.15  4.15  4.15  4.15  4.15  4.15
5         2.15  2.15  2.15  2.15  2.15  4.15  4.15  4.15  4.15  4.15
6         2.15  2.15  2.15  2.15  2.15  4.15  4.15  4.15  4.15  4.15
7         2.15  2.15  2.15  2.15  2.15  4.15  4.15  4.15  4.15  4.15
8         2.15  2.15  2.15  2.15  2.15  4.15  4.15  4.15  4.15  4.15
9         2.15  2.15  2.15  2.15  2.15  4.15  4.15  4.15  4.15  4.15

In [20]:
temp = !cat boot_stats.txt

for base_log in range(3,8):
    for level in range(3,10):
        #test = "keys/80_1024_1_%d_%d"%(base_log,level)
        test = "keys/std62_2048_1_%d_%d"%(base_log,level)
        if test not in [line.split()[0] for line in temp]:
            print("*"*80, "\n", test, "\n", "*"*80, "\n")
            !target/debug/test_boot $base_log $level 
            print("*"*80, "\n\n")
        

******************************************************************************** 
 keys/std62_2048_1_7_3 
 ******************************************************************************** 

base_log: 7
level: 3
key path: keys/std62_2048_1_7_3
loading LWE sk 0... 

x [4.0]

x* [4.0]
oooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo

v0 0.00000000000000000000000000000000000004701977403289152
n0 4
s0 60
loading BSK 00... 

f(x*) [4.012172191359497]
oooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo

v1 0.000000039775234957536064
n1 54
s1 10
keys/std62_2048_1_7_3 10 38 73531 11896
******************************************************************************** 


******************************************************************************** 
 keys/std62_2048_1_7_4 
 ******************************************************************************** 

base_log: 7
level: 4
key path: keys/std62_2048_1_7_4
loading LWE sk 0... 

x [4.0]

x* [4.0]
oooooooooooooooooooo

In [11]:
def splitter2(test):
    tmp = test.replace("NA","0").split()
    return tmp[0].split("/")[1].split("_") + [int(item) for item in tmp[1:]]

temp = !cat boot_stats.txt

[splitter2(line) for line in temp]
df2 = pd.DataFrame([splitter2(line) for line in temp], 
        columns=["bs","poly","dim","base_log","level","free", "sk_load", "bsk_load", "boot"])

df2.head()
                                                     

,bs,poly,dim,base_log,level,free,sk_load,bsk_load,boot
0,80,1024,1,5,3,5,13,22613,2893
1,80,1024,1,6,4,14,13,24416,3779
2,80,1024,1,3,3,0,19,18493,2880
3,80,1024,1,3,4,0,22,23930,3735
4,80,1024,1,3,6,8,16,35600,5067


In [12]:
df2.pivot_table(values="free", index="level", columns=["poly", "base_log"])


poly     1024                 2048            
base_log    3   4   5   6   7    3   4   5   6
level                                         
3           0   0   5   8  11    0   0   4   7
4           0   6  10  14  18    0   5   9  13
5           5  10  15  20  21    4   9  14  19
6           8  14  20  22  21    7  13  19  25
7          11  18  23  22  21   10  17  24  31
8          14  22  23  22  21   13  21  29  37
9          17  24  23  22  21   16  25  34  42

In [17]:
tmp = df2.pivot_table(values="sk_load", index="level", columns=["poly", "base_log"])
#print(tmp.mean().mean())
tmp


poly      1024                          2048                  
base_log     3     4     5     6     7     3     4     5     6
level                                                         
3         14.0   9.5  13.0   8.0  19.0  27.0  22.0  12.0   8.0
4         22.0  12.0  15.0  13.0   8.0  26.0  18.0  18.0  20.0
5         11.0   9.0  23.0   8.0  13.0  17.0  20.0  19.0  11.0
6         12.5  17.0  19.0  46.0  18.0  11.0   7.0  17.0  31.0
7         10.5  15.5  17.0  17.0   7.0   6.0  19.0  17.0  11.0
8          8.0  15.5  14.0  18.0  10.0  17.0  19.0  14.0  17.0
9          9.5  22.0  29.0  10.0   6.0  25.0  24.0  23.0  12.0

In [18]:
tmp = (df2.pivot_table(values="bsk_load", index="level", columns=["poly", "base_log"])/1000.).round(1)
#print(tmp.mean(axis=1).values)
tmp


poly      1024                           2048                     
base_log     3     4     5     6     7      3      4      5      6
level                                                             
3         18.5  18.5  22.6  18.0  19.6   72.9   73.8   76.0   73.5
4         24.1  23.9  23.6  24.4  24.1  100.7   95.3   97.4   94.5
5         29.7  29.4  29.8  30.7  30.8  122.0  122.3  127.8  121.0
6         35.6  36.2  36.0  46.5  38.0  146.0  149.4  157.2  157.4
7         41.4  43.5  52.5  42.6  44.9  170.9  170.7  168.5  176.7
8         47.7  49.4  48.7  62.4  48.9  194.9  194.3  194.9  212.1
9         53.1  55.4  56.4  62.3  58.6  223.9  246.8  225.7  226.3

In [19]:
tmp = (df2.pivot_table(values="boot", index="level", columns=["poly", "base_log"])/1000.).round(1)
print(tmp.mean(axis=1).values)
tmp

[ 6.9         8.56666667 10.27777778 11.97777778 13.68888889 15.43333333
 17.66666667]


poly     1024                      2048                  
base_log    3    4    5    6    7     3     4     5     6
level                                                    
3         3.0  3.0  2.9  2.9  3.0  11.7  11.9  11.7  12.0
4         3.8  3.7  3.7  3.8  3.7  14.6  14.6  15.0  14.2
5         4.4  4.4  4.4  4.6  4.4  17.6  17.5  17.7  17.5
6         5.1  5.2  5.1  5.2  5.3  20.7  20.6  20.2  20.4
7         5.9  6.1  5.8  5.9  5.9  23.3  23.5  23.4  23.4
8         6.6  6.8  6.7  6.9  6.5  26.2  26.5  26.1  26.6
9         7.3  7.3  7.7  7.4  7.7  29.4  31.4  29.9  30.9

In [59]:
np.exp(-np.log(2)/(24*7))

0.9996562360941261

5.625